# Convolutional Sequence to Sequence Learning

This notebook implements the model in:  
Gehring, J., Auli, M., Grangier, D., Yarats, D., and Dauphin, Y. N. 2017. Convolutional sequence to sequence learning. In Proceedings of the 34th International Conference on Machine Learning-Volume 70, 1243-1252. [arXiv:1705.03122](https://arxiv.org/abs/1705.03122).  

This model is based on an *Encoder-Decoder* framework, in which the encoder and the decoder are both CNNs.  
This model encodes all the information of the source sequence into the encoder's hidden states $\{h_1, h_2, \dots, h_{T_x} \}$, and allows the decoder (via an *attention mechanism*) to look at the entire hidden states in *every step* when generating the target sequence, instead of compressing all the information into a fixed-length context vector. This design aims to further relieve the *information compression*.  

![Conv Seq2Seq Learning](fig/conv-seq2seq-learning.png)

In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Preparing Data

In [2]:
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    """
    Tokenize German text. 
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenize English text.
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [3]:
from torchtext.data import Field, BucketIterator

# Set `batch_first=True` in the `Field`.
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', 
            lower=True, include_lengths=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', 
            lower=True, include_lengths=True, batch_first=True)

In [4]:
from torchtext.datasets import Multi30k

train_data, valid_data, test_data = Multi30k.splits(exts=['.de', '.en'], 
                                                    # fields=[SRC, TRG], 
                                                    fields=[('src', SRC), ('trg', TRG)], 
                                                    root='data/')

In [5]:
print(train_data[0].src)
print(train_data[0].trg)

['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']
['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']


In [6]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

len(SRC.vocab), len(TRG.vocab)

(7855, 5893)

In [7]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [8]:
for batch in train_iterator:
    batch_src, batch_src_lens = batch.src
    batch_trg, batch_trg_lens = batch.trg
    break
print(batch_src)
print(batch_src_lens)
print(batch_trg)
print(batch_trg_lens)

tensor([[  2,   5,  13,  ...,   1,   1,   1],
        [  2,   5,  13,  ...,   1,   1,   1],
        [  2,  43, 253,  ...,   1,   1,   1],
        ...,
        [  2,   5,  13,  ...,   1,   1,   1],
        [  2,  18,  30,  ...,   1,   1,   1],
        [  2,  18,   0,  ...,   1,   1,   1]])
tensor([14, 17, 12, 11, 17, 21, 12, 16, 14, 11, 23, 23,  8, 11,  9, 14, 19, 20,
        12, 16,  9, 11, 13, 20, 21, 29, 13, 22, 14, 16, 10,  9, 15, 12, 17, 10,
        14, 22, 17, 20, 23, 23, 12, 17, 15, 19, 17, 15, 16,  7, 14, 15, 16, 12,
        17, 14, 18, 18, 14, 14, 17, 21, 12, 12,  9, 19, 12, 14, 12, 11, 10, 13,
        18, 14,  9, 11, 10, 12, 10, 25, 14, 18, 15, 16, 15, 18, 13,  9, 21, 11,
        20, 12, 13, 14, 14, 17, 10, 13, 18, 30, 14, 12, 13,  9, 10, 15, 13, 10,
        12, 15, 13, 18, 17, 13, 11, 12, 10, 16, 12, 13, 24, 14, 19, 19, 10, 20,
        12, 11])
tensor([[   2,    4,    9,  ...,    1,    1,    1],
        [   2,    4,    9,  ...,    1,    1,    1],
        [   2,   48,   25,  .

## Build Model
### Encoder
* Use CNN instead of RNN.  
* Residual connection.  
* *Token Embeddings* + *Positional Embeddings*.  
* Convolutional Blocks
    * Add $(filter\_size - 1)/2$ padding elements on *each side* of the sequence.   
    * Use *GLU* (*Gated Linear Units*) as the activation function.  
        * GLU will half the hidden dimension.  
    * Residual connection  


GLU formula:  
$$
\mathrm{GLU}(a, b) = a \otimes \mathrm{sigmoid}(b)
$$
where $a$ and $b$ are inputs in the same shape, and $\otimes$ is element-wise product.  

In [9]:
class Encoder(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, n_layers, kernel_size, dropout, pad_idx, max_len=100):
        super().__init__()
        self.kernel_size = kernel_size
        self.scale = 0.5**0.5

        self.tok_emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.pos_emb = nn.Embedding(max_len, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)
        
        # The `kernel_size` must be odd. 
        self.convs = nn.ModuleList([nn.Conv1d(hid_dim, hid_dim*2, kernel_size=kernel_size, 
                                              padding=(kernel_size-1)//2) for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, mask):
        # src/mask: (batch, step)
        pos = torch.arange(src.size(1), device=src.device).repeat(src.size(0), 1)
        
        # Element-wise addition
        # embedded: (batch, step, emb_dim)
        embedded = self.tok_emb(src) + self.pos_emb(pos)
        embedded = self.dropout(embedded)

        # hidden: (batch, hid_dim=channels, step)
        hidden = self.emb2hid(embedded).permute(0, 2, 1)
        # Fill the padding positions with zeros, consistent with CNN padding. 
        # Fill the padding positions every time before passing to a conv-layer. 
        hidden.masked_fill_(mask.unsqueeze(1), 0)
        
        for i, conv in enumerate(self.convs):
            # conved: (batch, hid_dim*2, step)
            conved = conv(self.dropout(hidden))
            # `F.glu`: the `input` is split in half along `dim` to form `a` and `b`. 
            # conved: (batch, hid_dim, step)
            conved = F.glu(conved, dim=1)
            # Residual connection
            hidden = (hidden + conved) * self.scale
            # Fill the padding positions with zeros, consistent with CNN padding. 
            # Fill the padding positions every time before passing to a conv-layer. 
            hidden.masked_fill_(mask.unsqueeze(1), 0)

        # conv_outs: (batch, step, emb_dim)
        conv_outs = self.hid2emb(hidden.permute(0, 2, 1))
        conv_outs.masked_fill_(mask.unsqueeze(2), 0)
        # Residual connection
        # combined: (batch, step, emb_dim)
        combined = (embedded + conv_outs) * self.scale
        combined.masked_fill_(mask.unsqueeze(2), 0)

        return conv_outs, combined

In [10]:
SRC_IN_DIM = len(SRC.vocab)
TRG_IN_DIM = len(TRG.vocab)
EMB_DIM = 256
HID_DIM = 512
ENC_N_LAYERS = 10
DEC_N_LAYERS = 10
ENC_KERNEL_SIZE = 3 # Must be odd
DEC_KERNEL_SIZE = 3 # Can be even or odd
ENC_DROPOUT = 0.25
DEC_DROPOUT = 0.25
ENC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
DEC_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

encoder = Encoder(SRC_IN_DIM, EMB_DIM, HID_DIM, ENC_N_LAYERS, ENC_KERNEL_SIZE, 
                  ENC_DROPOUT, ENC_PAD_IDX).to(device)

mask = (batch_src == encoder.tok_emb.padding_idx)
enc_conv_outs, enc_combined = encoder(batch_src, mask)

print(batch_src.size())
print(enc_conv_outs.size())
print(enc_combined.size())

torch.Size([128, 30])
torch.Size([128, 30, 256])
torch.Size([128, 30, 256])


In [11]:
print(((enc_conv_outs == 0) == mask.unsqueeze(2)).all())
print(((enc_combined == 0) == mask.unsqueeze(2)).all())

tensor(True)
tensor(True)


### Decoder
* Use CNN instead of RNN.  
* Residual connection.  
* *Token Embeddings* + *Positional Embeddings*.  
* Convolutional Blocks
    * Add $filter\_size - 1$ padding elements at the *beginning* of the sequence.   
    * Use *GLU* (*Gated Linear Units*) as the activation function.  
        * GLU will half the hidden dimension.  
    * Residual connection  
* Attention  
    * Query: the embedding of current word  
    * Keys: the encoded representations - `conv_outs`  
        * `conv_outs` is good for getting a larger context over the encoded sequence  
    * Values: the encoded representations - `combined`  
        * `combined` has more information about the specific token and is thus more useful for prediction  

In [12]:
class Attention(nn.Module):
    """
    A dot-product attention. 
    """
    def __init__(self):
        super().__init__()

    def forward(self, queries, keys, values, mask):
        """
        Multi-step forward. 
        """
        # queries: (batch, trg_step, emb_dim)
        # keys: (batch, src_step, emb_dim)
        # values: (batch, src_step, emb_dim)
        # mask: (batch, src_step)

        # energy/attens: (batch, trg_step, src_step)
        energy = queries.bmm(keys.permute(0, 2, 1))
        energy.masked_fill_(mask.unsqueeze(1), -np.inf)

        # Attention along the source sequence
        attens = F.softmax(energy, dim=-1)

        # Apply the attention
        # attened_values: (batch, trg_step, emb_dim)
        return attens, attens.bmm(values)

In [13]:
attention = Attention()

# Use `combined` as an pseudo queries for checking. 
attens, attened_values = attention(enc_combined, enc_conv_outs, enc_combined, mask)
print(attens.size())
print(attened_values.size())

torch.Size([128, 30, 30])
torch.Size([128, 30, 256])


In [14]:
print(attens.sum(dim=-1))
print((attens == 0) == mask.unsqueeze(1))

tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
       grad_fn=<SumBackward1>)
tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [T

#### Is it necessary to fill the length padding positions with zeros?  
NO. Because the convolutional paddings are all at the sequence begining, so that the conv-layer would be forbidden to look at the token that it tries to predict. Hence, the elements in the length padding positions are also forbidden to pass to the valid (non-padding) positions in the upper layer. 

In [15]:
class Decoder(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, n_layers, kernel_size, dropout, pad_idx, max_len=100):
        super().__init__()
        self.kernel_size = kernel_size
        self.scale = 0.5**0.5

        self.tok_emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.pos_emb = nn.Embedding(max_len, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)

        self.attention = Attention()
        self.atten_emb2hid = nn.Linear(emb_dim, hid_dim)
        self.atten_hid2emb = nn.Linear(hid_dim, emb_dim)

        self.convs = nn.ModuleList([nn.Conv1d(hid_dim, hid_dim*2, 
                                              kernel_size=kernel_size) for _ in range(n_layers)])
        
        self.fc = nn.Linear(emb_dim, in_dim)
        self.dropout = nn.Dropout(dropout)


    def forward(self, trg, enc_conv_outs, enc_combined, mask):
        # trg: (batch, trg_step)
        pos = torch.arange(trg.size(1), device=trg.device).repeat(trg.size(0), 1)

        # Element-wise addition
        # embedded: (batch, trg_step, emb_dim)
        embedded = self.tok_emb(trg) + self.pos_emb(pos)
        embedded = self.dropout(embedded)
        
        # hidden: (batch, hid_dim=channels, trg_step)
        hidden = self.emb2hid(embedded).permute(0, 2, 1)
        # Note: For the decoder, NO need to fill the padding positions with zeros. 
        # begin_padding: (batch, hid_dim, kernel_size-1)
        begin_padding = torch.zeros(hidden.size(0), hidden.size(1), self.kernel_size-1, device=trg.device)

        for i, conv in enumerate(self.convs):
            # Manually pad zeros at the beginning. 
            padded_hidden = torch.cat([begin_padding, self.dropout(hidden)], dim=-1)
            # conved: (batch, hid_dim*2, trg_step)
            conved = conv(padded_hidden)
            # `F.glu`: the `input` is split in half along `dim` to form `a` and `b`. 
            # conved: (batch, hid_dim, step)
            conved = F.glu(conved, dim=1)

            # conved_projected: (batch, trg_step, emb_dim)
            conved_projected = self.atten_hid2emb(conved.permute(0, 2, 1))
            # attens: (batch, trg_step, src_step)
            # attened_values: (batch, trg_step, emb_dim)
            attens, attened_values = self.attention((embedded + conved_projected)*self.scale, 
                                                    enc_conv_outs, enc_combined, mask)

            # attened_projected: (batch, hid_dim, trg_step)
            attened_projected = self.atten_emb2hid(attened_values).permute(0, 2, 1)
            conved_attened = (conved + attened_projected) * self.scale

            # Residual connection
            # Note: For the decoder, NO need to fill the padding positions with zeros. 
            hidden = (hidden + conved_attened) * self.scale
            
        # conv_outs: (batch, trg_step, emb_dim)
        conv_outs = self.hid2emb(hidden.permute(0, 2, 1))
        # preds: (batch, trg_step, out_dim=in_dim)
        preds = self.fc(self.dropout(conv_outs))

        # Only attention at the top layer?
        return preds, attens

In [16]:
decoder = Decoder(TRG_IN_DIM, EMB_DIM, HID_DIM, DEC_N_LAYERS, DEC_KERNEL_SIZE, 
                  DEC_DROPOUT, DEC_PAD_IDX).to(device)

preds, attens = decoder(batch_trg, enc_conv_outs, enc_combined, mask)

print(batch_trg.size())
print(preds.size())
print(attens.size())

torch.Size([128, 29])
torch.Size([128, 29, 5893])
torch.Size([128, 29, 30])


### Seq2Seq

In [17]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        # src/mask: (batch, src_step)
        # trg: (batch, trg_step)
        mask = (src == self.encoder.tok_emb.padding_idx)
        enc_conv_outs, enc_combined = self.encoder(src, mask)
        # For the target sequence, the `<eos>` token should be sliced off before passing to the decoder. 
        # As there are no more tokens to be predicted after `<eos>`. 
        preds, attens = self.decoder(trg[:, :-1], enc_conv_outs, enc_combined, mask)

        # preds: (batch, trg_step-1, trg_out_dim)
        # attens: (batch, trg_step-1, src_step)
        return preds, attens

    def translate(self, src, src_lens, sos: int, trg_max_len: int=50):
        pass
        # Create an pseudo target sequence. 
        # pseudo_trg = torch.ones(trg_max_len, src.size(1), 
        #                         dtype=torch.long, device=src.device) * sos
        # preds, attens = self(src, src_lens, pseudo_trg, teacher_forcing_ratio=0)
        # return preds.argmax(dim=-1), attens

In [18]:
model = Seq2Seq(encoder, decoder).to(device)
preds, attens = model(batch_src, batch_trg)

print(batch_src.size())
print(batch_trg.size())
print(preds.size())
print(attens.size())

torch.Size([128, 30])
torch.Size([128, 29])
torch.Size([128, 28, 5893])
torch.Size([128, 28, 30])


## Train Model

In [19]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

encoder = Encoder(SRC_IN_DIM, EMB_DIM, HID_DIM, ENC_N_LAYERS, ENC_KERNEL_SIZE, 
                  ENC_DROPOUT, ENC_PAD_IDX).to(device)
decoder = Decoder(TRG_IN_DIM, EMB_DIM, HID_DIM, DEC_N_LAYERS, DEC_KERNEL_SIZE, 
                  DEC_DROPOUT, DEC_PAD_IDX).to(device)
model = Seq2Seq(encoder, decoder).to(device)

model.apply(init_weights)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 37,351,685 trainable parameters


In [20]:
# Initialize Embeddings 
ENC_UNK_IDX = SRC.vocab.stoi[SRC.unk_token]
DEC_UNK_IDX = TRG.vocab.stoi[TRG.unk_token]

model.encoder.tok_emb.weight.data[ENC_UNK_IDX].zero_()
model.encoder.tok_emb.weight.data[ENC_PAD_IDX].zero_()
model.decoder.tok_emb.weight.data[DEC_UNK_IDX].zero_()
model.decoder.tok_emb.weight.data[DEC_PAD_IDX].zero_()

print(model.encoder.tok_emb.weight[:5, :8])
print(model.decoder.tok_emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0140,  0.0106, -0.0217,  0.0019,  0.0008,  0.0056, -0.0079, -0.0081],
        [-0.0024,  0.0066,  0.0195,  0.0076, -0.0013, -0.0021, -0.0074, -0.0095],
        [-0.0058,  0.0025,  0.0031, -0.0048,  0.0008, -0.0035,  0.0121, -0.0095]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0120, -0.0037, -0.0044, -0.0082, -0.0085,  0.0148,  0.0086,  0.0026],
        [-0.0024, -0.0067,  0.0087,  0.0216, -0.0083, -0.0007,  0.0128,  0.0163],
        [ 0.0039, -0.0159, -0.0141, -0.0060,  0.0005, -0.0122, -0.0122,  0.0003]],
       grad_fn=<SliceBackward>)


In [21]:
loss_func = nn.CrossEntropyLoss(ignore_index=DEC_PAD_IDX, reduction='mean')
optimizer = optim.AdamW(model.parameters())

In [22]:
def train_epoch(model, iterator, optimizer, loss_func, clip):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        # Forward pass
        batch_src, batch_src_lens = batch.src
        batch_trg, batch_trg_lens = batch.trg
        # preds: (batch, trg_step-1, trg_out_dim)
        preds, _ = model(batch_src, batch_trg)
        
        # Calculate loss
        preds_flattened = preds.view(-1, preds.size(-1))
        batch_trg_flattened = batch_trg[:, 1:].flatten()
        loss = loss_func(preds_flattened, batch_trg_flattened)

        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update weights
        optimizer.step()
        # Accumulate loss
        epoch_loss += loss.item()
    return epoch_loss/len(iterator)

def eval_epoch(model, iterator, loss_func):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            batch_src, batch_src_lens = batch.src
            batch_trg, batch_trg_lens = batch.trg
            # preds: (batch, trg_step-1, trg_out_dim)
            preds, _ = model(batch_src, batch_trg)
            
            # Calculate loss
            preds_flattened = preds.view(-1, preds.size(-1))
            batch_trg_flattened = batch_trg[:, 1:].flatten()
            loss = loss_func(preds_flattened, batch_trg_flattened)
            
            # Accumulate loss and acc
            epoch_loss += loss.item()
    return epoch_loss/len(iterator)

In [23]:
import time
N_EPOCHS = 10
CLIP = 0.1
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss = train_epoch(model, train_iterator, optimizer, loss_func, CLIP)
    valid_loss = eval_epoch(model, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'models/tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')

KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('models/tut5-model.pt', map_location=device))

valid_loss = eval_epoch(model, valid_iterator, loss_func)
test_loss = eval_epoch(model, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')
print(f'Test Loss: {test_loss:.3f} |  Test PPL: {np.exp(test_loss):7.3f}')